## Installing packages 

In [1]:
! pip install pandas openpyxl
! pip install pyarrow
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 55.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 11.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import matplotlib.pyplot as plt

## Importing the files i prepared in the other notebooks

In [9]:

# Load the first CSV file into a DataFrame
tweets = pd.read_csv('tweets.csv')

# Load the second CSV file into a DataFrame
selected_loan_types = pd.read_csv('selected_loan_types.csv')

unadapted_borrowings = pd.read_csv('unadapted_borrowings.csv')


# Moving on to finding anglisiscms in a text

In [4]:
#preparing th eloans 
words_to_remove = {'e', 'I', 'm', 'k', 'b', 'a', 's', 'i', ',', 'en', 'for', 'og', 'de', 'af', 'et', 'man', 'her', 'da', 'over', 'end', 'have', 'mod', 'dig', 'alt', 'gang', 'to', 'igen', 'under', 'store', 'folk', 'gå', 'dog'}

# Ensure we are working with a copy
selected_loan_types = selected_loan_types.copy()

# 'Domestic Set' column
selected_loan_types.loc[:, 'Domestic Set'] = selected_loan_types['Domestic form'].apply(
    lambda x: set(str(x).split()) if isinstance(x, str) else set()
)

# Preprocess `Domestic` column to create sets of words for easier matching
# Handle NaN or non-string values

combined_loan_types = set().union(*selected_loan_types['Domestic Set'])

# Assuming combined_loan_types is a set
combined_loan_types = {item.lower() for item in combined_loan_types}

# Remove unwanted words from the combined_loan_types set
combined_loan_types -= {word.lower() for word in words_to_remove}

# Now combined_loan_types has the filtered words


In [11]:
print(tweets)

                                                   text  \
0                     35-årig mand erkender drab [LINK]   
1     BILLEDSERIE: Hjerteforeningens lokalkomite man...   
2       Det må jo være fraregnet #LordBendtner 😉 [LINK]   
3     Det er så godt, @USER! Psykiatriske disgnoser ...   
4     Nøddesbos hånd i Herning-rundkørsel: Jeg er en...   
...                                                 ...   
1042  @USER Jeg spørger bare, hvorfor dét er i orden...   
1043                                    Så ENIG! [LINK]   
1044  @USER Men det var jo ret sjovt :) som ikke er ...   
1045  DAT fejrer 30 år med fly Det er i dag 30 år si...   
1046  Eriksen har en glimrende statistik mod City [L...   

                                           Cleaned Text  \
0                     35-årig mand erkender drab [link]   
1     billedserie: hjerteforeningens lokalkomite man...   
2       det må jo være fraregnet #lordbendtner 😉 [link]   
3     det er så godt, @user! psykiatriske disgnoser ...

In [12]:

# Function to match and calculate
def find_matches(tokens, loan_types):
    matching_words = []
    matching_indices = []
    for idx, word in enumerate(tokens):
        if word in loan_types:
            matching_words.append(word)
            matching_indices.append(idx)  # Store index of the matching word
    return matching_words, matching_indices, len(matching_words)

# Handle cases where Tokens is not a string
results = []

for tokens in tweets['Tokens']:
    # Check if tokens is a string; if not, convert to empty string
    if isinstance(tokens, str):
        tokens = tokens.lower().split()  # Lowercase and split by spaces
    else:
        tokens = []  # Treat non-strings (e.g., NaN or floats) as empty list

    # Match tokens against combined_loan_types
    matches, match_indices, count = find_matches(tokens, combined_loan_types)
    results.append({'Matching Words': matches, 'Match Indices': match_indices, 'Count': count})

# Add results to DataFrame
tweets['matches'] = [result['Matching Words'] for result in results]
tweets['match_indices'] = [result['Match Indices'] for result in results]
tweets['match_count'] = [result['Count'] for result in results]


In [13]:
print(tweets)

                                                   text  \
0                     35-årig mand erkender drab [LINK]   
1     BILLEDSERIE: Hjerteforeningens lokalkomite man...   
2       Det må jo være fraregnet #LordBendtner 😉 [LINK]   
3     Det er så godt, @USER! Psykiatriske disgnoser ...   
4     Nøddesbos hånd i Herning-rundkørsel: Jeg er en...   
...                                                 ...   
1042  @USER Jeg spørger bare, hvorfor dét er i orden...   
1043                                    Så ENIG! [LINK]   
1044  @USER Men det var jo ret sjovt :) som ikke er ...   
1045  DAT fejrer 30 år med fly Det er i dag 30 år si...   
1046  Eriksen har en glimrende statistik mod City [L...   

                                           Cleaned Text  \
0                     35-årig mand erkender drab [link]   
1     billedserie: hjerteforeningens lokalkomite man...   
2       det må jo være fraregnet #lordbendtner 😉 [link]   
3     det er så godt, @user! psykiatriske disgnoser ...

In [15]:
total = tweets['match_count'].sum()



print(total)

580


In [16]:
# nice! now we have a list of marked enlgish loans we can run the models on#
tweets.to_csv('combined_loan_types_annotation.csv', index=False)

## Applying the code to only the direct borrowings 

In [8]:
# Ensure we are working with a copy
unadapted_borrowings = unadapted_borrowings.copy()

# Safely create the 'Domestic Set' column
unadapted_borrowings.loc[:, 'Domestic Set'] = unadapted_borrowings['Domestic form'].apply(
    lambda x: set(str(x).split()) if isinstance(x, str) else set()
)
# Preprocess `Domestic` column to create sets of words for easier matching
# Handle NaN or non-string values

In [9]:
unadapted_borrowings = set().union(*unadapted_borrowings['Domestic Set'])

In [10]:
# Assuming combined_loan_types is a set
unadapted_borrowings = {item.lower() for item in unadapted_borrowings}

In [11]:
# Function to match and calculate
def find_matches(tokens, loan_types):
    matching_words = [word for word in tokens if word in loan_types]
    return matching_words, len(matching_words)

In [12]:
# Handle cases where Tokens is not a string
results = []

for tokens in df['Tokens']:
    # Check if tokens is a string; if not, convert to empty string
    if isinstance(tokens, str):
        tokens = tokens.lower().split()  # Lowercase and split by spaces
    else:
        tokens = []  # Treat non-strings (e.g., NaN or floats) as empty list

    # Match tokens against combined_loan_types
    matches, count = find_matches(tokens, unadapted_borrowings)
    results.append({'Matching Words': matches, 'Count': count})

# Add results to DataFrame
df['matches'] = [result['Matching Words'] for result in results]
df['match_count'] = [result['Count'] for result in results]


In [13]:
print(df[['Tokens', 'matches', 'match_count']])

                                                 Tokens  \
0                            35 årig mand erkender drab   
1     billedserie hjerteforeningens lokalkomite mang...   
2                 det må jo være fraregnet lordbendtner   
3     det er så godt , ! psykiatriske disgnoser skal...   
4     nøddesbos hånd i herning rundkørsel jeg er en ...   
...                                                 ...   
1042  jeg spørger bare , hvorfor dét er i orden , nå...   
1043                                          så enig !   
1044  men det var jo ret sjovt ) som ikke er ond mod...   
1045  dat fejrer 30 år med fly det er i dag 30 år si...   
1046  eriksen har en glimrende statistik mod city pl...   

                                    matches  match_count  
0                                        []            0  
1                                     [for]            1  
2                                        []            0  
3                                   [,, en]            

In [14]:
# List of words to remove
words_to_remove = {'e', 'I', 'm', 'k', 'b', 'a', 's', 'i', ',', 'en', 'for',  'og', 'de', 'af', 'et', 'man', 'her', 'da', 'over', 'end', 'have', 'mod', 'dig', 'alt', 'gang', 'to', 'igen', 'under', 'store', 'folk', 'gå', 'dog'}

# Remove unwanted words from the 'matches' column
df['matches'] = df['matches'].apply(lambda match_list: [word for word in match_list if word.lower() not in words_to_remove])

# Update the 'match_count' column with the new count after filtering
df['match_count'] = df['matches'].apply(len)

# Flatten the matches column into a single list for the summary table
all_matches = [match for sublist in df['matches'] for match in sublist]

# Create a frequency table (akin to R's table)
match_counts = pd.Series(all_matches).value_counts()

# Convert to a DataFrame for tabular output
summary_table = match_counts.reset_index()
summary_table.columns = ['Match', 'Count']

# Display the updated DataFrame and summary table
print(df[['Tokens', 'matches', 'match_count']])
print(summary_table)

# Total match count: 616


                                                 Tokens    matches  \
0                            35 årig mand erkender drab         []   
1     billedserie hjerteforeningens lokalkomite mang...         []   
2                 det må jo være fraregnet lordbendtner         []   
3     det er så godt , ! psykiatriske disgnoser skal...         []   
4     nøddesbos hånd i herning rundkørsel jeg er en ...         []   
...                                                 ...        ...   
1042  jeg spørger bare , hvorfor dét er i orden , nå...  [podcast]   
1043                                          så enig !         []   
1044  men det var jo ret sjovt ) som ikke er ond mod...         []   
1045  dat fejrer 30 år med fly det er i dag 30 år si...         []   
1046  eriksen har en glimrende statistik mod city pl...     [city]   

      match_count  
0               0  
1               0  
2               0  
3               0  
4               0  
...           ...  
1042            1  

In [15]:
#saving  

df.to_csv('unadapted_borrowings_annotated.csv', index=False)
